In [1]:
import numpy as np
import pandas as pd
from pypfopt import EfficientFrontier,risk_models,expected_returns,objective_functions
from pypfopt.base_optimizer import BaseConvexOptimizer
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import statsmodels.api as sm

Problem 1

In [2]:
ff = pd.read_excel('ap-2023-ff-data.xlsx')
ff['Year-Month'] = ff['Year'].astype(str) + '-' + ff['Month'].astype(str)
ff['Year-Month'] = pd.to_datetime(ff['Year-Month'], format='%Y-%m')
ff.set_index('Year-Month', inplace=True)
ff.drop(['Year', 'Month'], axis=1, inplace=True)

ami_day_ret = pd.read_excel('ap-2023-amihud-data.xlsx',sheet_name = 'returns-daily')
ami_day_ret['Date'] = pd.to_datetime(ami_day_ret['Date'], format='%Y-%m-%d')
ami_day_ret.set_index('Date', inplace=True)

ami_dvd = pd.read_excel('ap-2023-amihud-data.xlsx',sheet_name = 'dollar-vols-daily')
ami_dvd['Date'] = pd.to_datetime(ami_dvd['Date'], format='%Y-%m-%d')
ami_dvd.set_index('Date', inplace=True)

ami_mon_ret = pd.read_excel('ap-2023-amihud-data.xlsx',sheet_name = 'returns-monthly')
ami_mon_ret['Year-Month'] = ami_mon_ret['Year'].astype(str) + '-' + ami_mon_ret['Month'].astype(str)
ami_mon_ret['Year-Month'] = pd.to_datetime(ami_mon_ret['Year-Month'], format='%Y-%m')
ami_mon_ret.set_index('Year-Month', inplace=True)
ami_mon_ret.drop(['Year', 'Month'], axis=1, inplace=True)

ami_ff = pd.read_excel('ap-2023-amihud-data.xlsx',sheet_name = 'ff-factors')
ami_ff['Year-Month'] = ami_ff['Year'].astype(str) + '-' + ami_ff['Month'].astype(str)
ami_ff['Year-Month'] = pd.to_datetime(ami_ff['Year-Month'], format='%Y-%m')
ami_ff.set_index('Year-Month', inplace=True)
ami_ff.drop(['Year', 'Month'], axis=1, inplace=True)

In [3]:
ff3f = ff[['Mkt-RF', 'SMB', 'HML']]

In [4]:
mu = expected_returns.mean_historical_return(ff3f, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff3f, log_returns=True,frequency=252,returns_data = True)

rf = 0

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

[0.37   0.1462 0.4838]
Optimal Portfolio Return: 0.9638700369190636
Optimal Portfolio Risk: 0.3297476496654774
Optimal Sharpe Ratio: 2.923053546846781


In [5]:
Xb = ff[['Mkt-RF', 'SMB', 'HML']]
yb = ff['RMW']
Xb = sm.add_constant(Xb)
modelb = sm.OLS(yb, Xb).fit()
print(modelb.summary())

                            OLS Regression Results                            
Dep. Variable:                    RMW   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     36.35
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           8.01e-22
Time:                        19:26:28   Log-Likelihood:                 1713.5
No. Observations:                 696   AIC:                            -3419.
Df Residuals:                     692   BIC:                            -3401.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0034      0.001      4.308      0.0

In [6]:
Xc = ff[['Mkt-RF', 'SMB', 'HML']]
yc = ff['CMA']
Xc = sm.add_constant(Xc)
modelc = sm.OLS(yc, Xc).fit()
print(modelc.summary())

                            OLS Regression Results                            
Dep. Variable:                    CMA   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     236.6
Date:                Fri, 10 Nov 2023   Prob (F-statistic):          1.23e-105
Time:                        19:26:28   Log-Likelihood:                 1986.4
No. Observations:                 696   AIC:                            -3965.
Df Residuals:                     692   BIC:                            -3947.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.001      4.117      0.0

In [7]:
ff5f = ff[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

In [8]:
mu = expected_returns.mean_historical_return(ff5f, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff5f, log_returns=True,frequency=252,returns_data = True)

rf = 0

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

[0.1708 0.118  0.     0.315  0.3962]
Optimal Portfolio Return: 0.8120344598560627
Optimal Portfolio Risk: 0.16365671493808967
Optimal Sharpe Ratio: 4.961815713844985


In [9]:
Xe = ff[['Mkt-RF', 'SMB', 'RMW','CMA']]
ye = ff['HML']
Xe = sm.add_constant(Xe)
modele = sm.OLS(ye, Xe).fit()
print(modele.summary())

                            OLS Regression Results                            
Dep. Variable:                    HML   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     153.1
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           9.62e-94
Time:                        19:26:28   Log-Likelihood:                 1695.2
No. Observations:                 696   AIC:                            -3380.
Df Residuals:                     691   BIC:                            -3358.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.001     -1.196      0.2

In [10]:
Xf = ff[['Mkt-RF', 'SMB','HML', 'RMW','CMA']]
yf = ff['MOM']
Xf = sm.add_constant(Xf)
modelf = sm.OLS(yf, Xf).fit()
print(modelf.summary())

                            OLS Regression Results                            
Dep. Variable:                    MOM   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     16.77
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           1.17e-15
Time:                        19:26:28   Log-Likelihood:                 1255.4
No. Observations:                 696   AIC:                            -2499.
Df Residuals:                     690   BIC:                            -2472.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.002      4.463      0.0

In [11]:
Xg = ff[['Mkt-RF', 'SMB','HML', 'RMW','CMA']]
yg = ff['STREV']
Xg = sm.add_constant(Xg)
modelg = sm.OLS(yg, Xg).fit()
print(modelg.summary())

                            OLS Regression Results                            
Dep. Variable:                  STREV   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     19.19
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           6.58e-18
Time:                        19:26:28   Log-Likelihood:                 1466.3
No. Observations:                 696   AIC:                            -2921.
Df Residuals:                     690   BIC:                            -2893.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0036      0.001      3.099      0.0

In [12]:
mu = expected_returns.mean_historical_return(ff, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff, log_returns=True,frequency=252,returns_data = True)

rf = 0

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

[0.1197 0.0712 0.     0.2159 0.3161 0.1342 0.143 ]
Optimal Portfolio Return: 0.9678422209450578
Optimal Portfolio Risk: 0.15180144311308222
Optimal Sharpe Ratio: 6.375711594678834


In [13]:
ff_in= ff[ff.index <=  '1999-12-31']
ff_out = ff[ff.index >=  '1999-12-31']

In [14]:
mu = expected_returns.mean_historical_return(ff_in, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff_in, log_returns=True,frequency=252,returns_data = True)

rf = 0

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

[0.0576 0.0498 0.1016 0.2087 0.1877 0.1929 0.2017]
Optimal Portfolio Return: 1.1548526071252876
Optimal Portfolio Risk: 0.12778908844028827
Optimal Sharpe Ratio: 9.037176970433693


In [15]:
mu = expected_returns.mean_historical_return(ff_out, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff_out, log_returns=True,frequency=252,returns_data = True)

rf = 0

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

[0.1931 0.1649 0.     0.3345 0.1949 0.0713 0.0414]
Optimal Portfolio Return: 0.9805402314976324
Optimal Portfolio Risk: 0.18400373084770424
Optimal Sharpe Ratio: 5.328914946345319


Problem 2

In [67]:
ami_ILIIQ = pd.DataFrame()
ami_ILIIQ = 10**6 * np.abs(ami_day_ret)/ami_dvd #calculate ILIIQ

In [77]:
ami_ILIIQ

,10147,10890,11850,11891,15720,16600,17144,17750,17778,17929,...,49015,49154,49905,52038,52230,52695,52898,53065,60442,61313
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-02,0.003925,0.014231,0.000045,0.003617,0.000446,0.002063,0.000741,0.000217,0.001695,0.007090,...,0.001292,0.000802,0.002526,0.000555,0.011638,0.000000,0.087295,0.000631,0.000266,0.046388
1996-01-03,0.002310,0.007167,0.000027,0.003820,0.000214,0.002892,0.001012,0.000059,0.001789,0.012407,...,0.000732,0.000488,0.000862,0.000392,0.002516,0.000711,0.061254,0.001165,0.000563,0.000000
1996-01-04,0.000659,0.010572,0.000033,0.002310,0.000368,0.001684,0.000975,0.000202,0.001669,0.007852,...,0.003026,0.000261,0.000280,0.000351,0.004274,0.002786,0.036124,0.003204,0.000207,0.006007
1996-01-05,0.000000,0.010155,0.000146,0.007314,0.000364,0.002690,0.000845,0.000015,0.001763,0.000000,...,0.001239,0.000403,0.000000,0.002949,0.011388,0.000000,0.000000,0.000827,0.000117,0.010229
1996-01-08,0.001221,0.012317,0.000226,0.004513,0.000908,0.004415,0.000267,0.000089,0.000537,0.037522,...,0.002341,0.001702,0.004717,0.001559,0.011107,0.005243,0.112708,0.002548,0.001041,0.109606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,0.000016,0.002887,0.000023,0.000021,0.000000,0.000008,0.000046,0.000016,0.000657,0.000393,...,0.000057,0.000081,0.000432,0.000155,0.000157,0.000012,0.000845,0.000167,0.000010,0.000144
2015-12-28,0.000038,0.001772,0.000010,0.000088,0.000012,0.000017,0.000017,0.000045,0.000098,0.000060,...,0.000021,0.000023,0.001050,0.000057,0.000140,0.000063,0.000710,0.000046,0.000034,0.000416
2015-12-29,0.000012,0.000660,0.000008,0.000231,0.000088,0.000121,0.000040,0.000050,0.000011,0.000518,...,0.000180,0.000064,0.001101,0.000005,0.000296,0.000075,0.000048,0.000189,0.000089,0.000347


In [68]:
ami_ILIIQ_month = ami_ILIIQ.resample('M').mean() #resample monthly to get monthly ILIIQ
ami_ILIIQ_month.index = ami_ILIIQ_month.index.to_period('M').to_timestamp()

In [69]:
#create new dataframe to store the list of top/bottom liquid stock list
ami_liquidity = pd.DataFrame(index=ami_ILIIQ_month.index, columns=['IL', 'L','IML']) 
for i in range(len(ami_ILIIQ_month)):
    IL = []
    L = []
    for j in range(len(ami_ILIIQ_month.columns)):
        if ami_ILIIQ_month.iloc[i, j] >= ami_ILIIQ_month.iloc[i].quantile(0.8):
            IL.append(ami_ILIIQ_month.columns[j])
        elif ami_ILIIQ_month.iloc[i, j] <= ami_ILIIQ_month.iloc[i].quantile(0.2):
            L.append(ami_ILIIQ_month.columns[j])
    ami_liquidity.at[ami_ILIIQ_month.index[i], 'IL'] = IL
    ami_liquidity.at[ami_ILIIQ_month.index[i], 'L'] = L

In [70]:
for i in range(1,len(ami_liquidity)): #calculate IML
    IL_mean = ami_ILIIQ_month.loc[ami_ILIIQ_month.index[i], ami_liquidity['IL'].iloc[i-1]].mean()
    L_mean = ami_ILIIQ_month.loc[ami_ILIIQ_month.index[i],ami_liquidity['L'].iloc[i-1]].mean()
    ami_liquidity.at[ami_liquidity.index[i], 'IML'] = IL_mean - L_mean
ami_liquidity['IML'] =  ami_liquidity['IML'].astype(float)

In [71]:
ami_liquidity

,IL,L,IML
Date,,,
1996-01-01,"[10890, 11891, 17929, 20204, 20415, 23393, 528...","[11850, 15720, 17750, 18411, 19393, 22592, 240...",NaN
1996-02-01,"[10890, 17929, 20204, 20415, 23393, 52230, 528...","[11850, 17750, 19350, 19393, 22592, 24010, 491...",0.018251
1996-03-01,"[17929, 20204, 20415, 23393, 40440, 52230, 528...","[11850, 17144, 17750, 18411, 19350, 19393, 225...",0.015029
1996-04-01,"[17778, 17929, 20204, 20415, 23393, 52230, 528...","[10147, 11850, 17144, 17750, 19393, 22592, 491...",0.018679
1996-05-01,"[11891, 17929, 20204, 20415, 23393, 52230, 528...","[10147, 11850, 17750, 18411, 19350, 19393, 225...",0.016740
...,...,...,...
2015-08-01,"[10890, 17929, 20204, 40440, 44951, 49905, 528...","[10147, 11850, 18411, 19350, 19393, 22592, 491...",0.000856
2015-09-01,"[10890, 17929, 20204, 40440, 44951, 49905, 528...","[10147, 11850, 18411, 19350, 19393, 22592, 491...",0.001054
2015-10-01,"[10890, 17929, 20204, 40440, 44951, 49905, 528...","[10147, 11850, 17144, 17750, 18411, 19393, 225...",0.001050


In [72]:
IML_mean = ami_liquidity.IML.mean() #(a)
IML_vol = ami_liquidity.IML.std()
IML_sharpe = IML_mean/IML_vol
print(IML_mean,IML_vol,IML_sharpe)

0.004286339752729787 0.005132140916183093 0.8351952572490252


In [73]:
IML_mean = ami_liquidity[ami_liquidity.index <= '2005-12-31'].IML.mean()
IML_vol = ami_liquidity[ami_liquidity.index <= '2005-12-31'].IML.std()
IML_sharpe = IML_mean/IML_vol
print(IML_mean,IML_vol,IML_sharpe)

0.007293838187437195 0.005600669637536531 1.3023153764601276


In [74]:
IML_mean = ami_liquidity[ami_liquidity.index > '2005-12-31'].IML.mean()
IML_vol = ami_liquidity[ami_liquidity.index > '2005-12-31'].IML.std()
IML_sharpe = IML_mean/IML_vol
print(IML_mean,IML_vol,IML_sharpe)

0.0013039038049782725 0.0018865615652012627 0.6911535934101197


In [75]:
ami_liquidity['IML_Invest'] = (1 +ami_liquidity.IML).cumprod() #(b)
print(ami_liquidity['IML_Invest'][-1])

2.770848558875869


In [76]:
Xd2 = ami_ff.iloc[1:]
yd2 = ami_liquidity['IML'].iloc[1:]
Xd2 = sm.add_constant(Xd2)
modeld2 = sm.OLS(yd2, Xd2).fit()
print(modeld2.summary())

                            OLS Regression Results                            
Dep. Variable:                    IML   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.3943
Date:                Fri, 10 Nov 2023   Prob (F-statistic):              0.813
Time:                        19:44:18   Log-Likelihood:                 922.24
No. Observations:                 239   AIC:                            -1834.
Df Residuals:                     234   BIC:                            -1817.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0043      0.000     12.455      0.0